In [ ]:
# Import the required packeages
import datetime
import os
from pyprediktormapclient.opc_ua import OPC_UA
from pyprediktormapclient.model_index import ModelIndex
from pyprediktormapclient.auth_client import AUTH_CLIENT
from dotenv import load_dotenv 
from pathlib import Path

In [ ]:
# Import Analytics Helper
from pyprediktormapclient.analytics_helper import AnalyticsHelper

# Import "Dataframer" Tools
from pyprediktormapclient.shared import *

In [ ]:
# Obtain the envrionment variables from .env file
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

In [ ]:
# Connecting to ModelIndex APIs 
model_data = ModelIndex(url=model_index_url, auth_client=auth_client, session=auth_client.session)

### Download data from modelindex api

In [ ]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

In [ ]:
# Types of Objects
object_types_json = model_data.get_object_types()
object_types = AnalyticsHelper(object_types_json)
object_types.dataframe

In [ ]:
namespace_list = object_types.namespaces_as_list(namespaces)

# Initating the OPC UA API with a fixed namespace list
opc_data = OPC_UA(rest_url=opcua_rest_url, opcua_url=opcua_server_url, namespaces=namespace_list, auth_client=auth_client)

In [ ]:
# Unique types of Objects
object_types_unique = object_types.dataframe[["Id", "Name"]].drop_duplicates()
object_types_unique

In [ ]:
# To get typeId by type name of an object
object_type_id = model_data.get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
# To get the objects of a type
sites_json = model_data.get_objects_of_type("SiteType")

# Send the returned JSON into a normalizer to get Id, Type, Name, Props and Vars as columns
sites = AnalyticsHelper(sites_json)
sites.list_of_names()

In [ ]:
# Analytics helper
sites.variables_as_dataframe()

In [ ]:
sites.list_of_ids()

In [ ]:
# Selecting the single site
site_id = sites.list_of_ids()[1]
site_id

In [ ]:
# Get all stringsets for one park
string_sets_for_first_park_as_json = model_data.get_object_descendants(
    "StringSetType", [site_id], "PV_Assets"
)
string_sets_for_first_park = AnalyticsHelper(string_sets_for_first_park_as_json)
string_sets_for_first_park.dataframe

In [ ]:
# Ancestors of an object type, get all trackers that are ancestor of the parks string sets

trackers_as_json = model_data.get_object_ancestors(
    "TrackerType", string_sets_for_first_park.list_of_ids(), "PV_Serves"
)
trackers = AnalyticsHelper(trackers_as_json)
trackers.variables_as_dataframe()

### Download data from the opc ua api

In [ ]:
# Live value data of trackers
live_value = opc_data.get_values(
    trackers.variables_as_list(["AngleMeasured", "AngleSetpoint"])
)
live_value

In [ ]:
# Historic value data of trackers, 1 days worth of data 30 days ago
one_day_historic_tracker_data = opc_data.get_historical_aggregated_values(
    start_time=(datetime.datetime.now() - datetime.timedelta(30)),
    end_time=(datetime.datetime.now() - datetime.timedelta(29)),
    pro_interval=3600000,
    agg_name="Average",
    variable_list=trackers.variables_as_list(["AngleMeasured"]),
)
one_day_historic_tracker_data

##### Ory Authentication

In [ ]:
import requests
import logging
from pydantic import AnyUrl, validate_call
from typing import Literal

In [ ]:
from pydantic import BaseModel, AnyUrl, validate_call, AwareDatetime, field_validator
from pyprediktormapclient.shared import request_from_api
import datetime
import json
import re

In [ ]:
class Ory_Login_Structure(BaseModel):
    method: str
    identifier: str
    password: str

In [ ]:
class Token(BaseModel):
    access_token: str
    expires_at: AwareDatetime = None
    
    @field_validator('expires_at', mode='before')
    def remove_nanoseconds(cls, v):
        if v is None:
            return v
        elif re.match(r"(\d\d\d\d)-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d).\d+(\S+)", v):
            return datetime.datetime.strptime(f"{v[:-11]}.+00:00", "%Y-%m-%dT%H:%M:%S.%z")
        return v

In [ ]:
rest_url = ory_url
headers = {"Content-Type": "application/json"}

In [ ]:
def request_from_api(
    rest_url: AnyUrl,
    method: Literal["GET", "POST"],
    endpoint: str,
    data: str = None,
    params: dict = None,
    headers: dict = None,
    extended_timeout: bool = False,
) -> str:
    """Function to perform the request to the ModelIndex server

    Args:
        rest_url (str): The URL with trailing shash
        method (str): "GET" or "POST"
        endpoint (str): The last part of the url (without the leading slash)
        data (str): defaults to None but can contain the data to send to the endpoint
        headers (str): default to None but can contain the headers og the request
    Returns:
        JSON: The result if successfull
    """
    request_timeout = (3, 300 if extended_timeout else 27)
    combined_url = f"{rest_url}{endpoint}"

    result = None
    if method == "GET":
        result = requests.get(combined_url, timeout=request_timeout, params=params, headers=headers)

    if method == "POST":
        result = requests.post(
            combined_url, data=data, headers=headers, timeout=request_timeout, params=params
        )
    
    result.raise_for_status()

    if 'application/json' in result.headers.get('Content-Type', ''):
        return result.json()

    else:
        logging.warning(f"Non-JSON response received from {combined_url}")
        return {"error": "Non-JSON response", "content": result.text}

In [ ]:
content = request_from_api(
    rest_url=rest_url,
    method="GET",
    endpoint="self-service/login/api",
    headers=headers,
    extended_timeout=True,
)

In [ ]:
def get_login_id(content):
    """Request login token from Ory"""
    print("Getting login ID...")
    
    print(f"Response for login ID: {content}")

    if "error" in content:
        # Handle the error appropriately
        raise RuntimeError(content["error"])

    if content.get("Success") is False or not isinstance(content.get("id"), str):
        error_message = content.get("ErrorMessage", "Unknown error occurred during login.")
        raise RuntimeError(error_message)

    id = content.get("id")
    return id

In [ ]:
id = get_login_id(content)
id

In [ ]:
params = {"flow": id}
body = (Ory_Login_Structure(method="password", identifier=username, password=password).model_dump())

In [ ]:
content = request_from_api(
    rest_url=rest_url,
    method="POST",
    endpoint="self-service/login",
    data=json.dumps(body),
    params=params,
    headers=headers,
    extended_timeout=True,
)

In [ ]:
def get_login_token(content) -> None:
    """Request login token from Ory"""
    print("Getting login token...")
    
    print(f"Response for login token: {content}")

    if content.get("Success") is False:
        raise RuntimeError(content.get("ErrorMessage"))
    
    # Return if no content from server
    if not isinstance(content.get("session_token"), str):
        raise RuntimeError(content.get("ErrorMessage"))
    token = Token(access_token=content.get("session_token"))

    # Check if token has expiry date, save it if it does
    if isinstance(content.get("session").get("expires_at"), str):
        # String returned from ory has to many chars in microsec. Remove them
        #from_string = content.get("session").get("expires_at")
        #date_object = datetime.datetime.strptime(f"{from_string[:-11]}.+00:00", "%Y-%m-%dT%H:%M:%S.%z")
        try:
            token = Token(access_token=token.access_token, expires_at=content.get("session").get("expires_at"))
        except Exception:
            # If string returned from Ory cant be parsed, still should be possible to use Ory,
            #  might be a setting in Ory to not return expiry date
            token = Token(access_token=token.access_token)

In [ ]:
token = get_login_token(content)
token

In [ ]:
def check_if_token_has_expired() -> bool:
    """Check if token has expired
    """
    if token is None or token.expires_at is None:
        return True

    return datetime.datetime.utcnow() > token.expires_at

In [ ]:
check_if_token_has_expired()

In [ ]:
model_data = ModelIndex(url=model_index_url, auth_client=token)
model_data

In [ ]:
def get_namespace_array() -> str:
    """Get the namespace array

    Returns:
        str: the JSON returned from the server
    """
    content = request_from_api(model_index_url, "GET", "query/namespace-array")

    return content

In [ ]:
# Listed sites on the model index api server
namespaces = model_data.get_namespace_array()
namespaces

In [ ]:
def request_new_ory_token() :
    """Request Ory token"""
    print("Requesting new ORY token...")
    get_login_id()
    get_login_token()
    print("New ORY token requested.")